In [38]:
# draft_2 - test
# import:
import random
from itertools import product
random.seed(1)

# indexing:
STARTING = 0
DESTINATION = 1
ROW = 0
COLUMN = 1

# variables:
n_rows = 1
n_cols = 10
list_position = [(0,0), (0,9)]
num_of_pedestrian = 3
sl_coverage = 2
md_coverage = 3
brightness_level_lowest = 1
brightness_level_highest = 3
brightness_level = list(range(brightness_level_lowest, brightness_level_highest + 1))

class PedestriansMaker():
    
    def __init__(self, n_rows, n_cols, list_position, num_of_pedestrian):
        self.list_position = list_position
        self.num_of_pedestrian = num_of_pedestrian
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.list_sd = self.create_starting_destination(self.list_position)
        self.list_pedestrian_sd = self.create_pedestrians_sd(self.num_of_pedestrian, self.list_sd)
        self.list_pedestrian_s = self.list_pedestrian_sd[STARTING]
        self.list_pedestrian_d = self.list_pedestrian_sd[DESTINATION]
        self.grid_pedestrian = self.create_location_grid(self.n_rows, self.n_cols, self.num_of_pedestrian, self.list_pedestrian_s)
        self.list_pedestian_pathway = self.create_pathway(self.list_pedestrian_s, self.num_of_pedestrian)
  
    @staticmethod
    def create_starting_destination(list_position):
        starting_point = list_position # since list mentioned in the init method, can we put self.list_position?
        destination_point = list_position
        list_sd = []
        for starting, destination in product(starting_point, destination_point):
            if starting != destination:
                list_sd.append((starting, destination))
        return list_sd # return a list of tuple [starting destination option]

    @staticmethod
    def create_pedestrians_sd(num_of_pedestrian, list_sd):
        pedestrian_s = []
        pedestrian_d = []
        for idx_pedestrian_starting in range(num_of_pedestrian):
            pedestrian_sd = random.choice(list_sd)
            pedestrian_s.append(pedestrian_sd[STARTING])
            pedestrian_d.append(pedestrian_sd[DESTINATION])
            list_pedestrian_sd = [pedestrian_s, pedestrian_d]
        return list_pedestrian_sd # return list of tuple [starting][destination]

    @staticmethod
    def create_location_grid(n_rows, n_cols, num_of_pedestrian, list_pedestrian_s):
        grid_pedestrian = []
        for idx_row in range(n_rows):
            row = []
            for idx_column in range(n_cols):
                row.append(0)
            grid_pedestrian.append(row)
        for idx_pedestrian in range(num_of_pedestrian):
            pedestrian_s = list_pedestrian_s[idx_pedestrian]
            grid_pedestrian[pedestrian_s[ROW]][pedestrian_s[COLUMN]] += 1
        return grid_pedestrian # return the grid and each pedestrian position on the grid
    
    @staticmethod
    def create_pathway(list_pedestrian_s, num_of_pedestrian):
        list_pedestrian_pathway = []
        for idx_pedestrian in range(num_of_pedestrian):
            pedestrian_s = list_pedestrian_s[idx_pedestrian]
            if pedestrian_s == (0,0): # will later generalize this
                row_update = 0
                col_update = 1
            else:
                row_update = 0
                col_update = -1
            list_pedestrian_pathway.append((row_update, col_update))
        return list_pedestrian_pathway # return the direction of a pedestrian

    @staticmethod
    def pedestrian_loc_update(list_current_pedestrian, list_pedestrian_pathway):
        updated_pedestrian_loc = []
        for idx_pedestrian in range(len(list_pedestrian_pathway)):
            pedestrian_loc = list_current_pedestrian[idx_pedestrian]
            pedestrian_row = pedestrian_loc[ROW]
            pedestrian_col = pedestrian_loc[COLUMN]
            pedestrian_pathway = list_pedestrian_pathway[idx_pedestrian]
            pedestrian_pathway_row = pedestrian_pathway[ROW]
            pedestrian_pathway_col = pedestrian_pathway[COLUMN]
            update_pedestrian_row = pedestrian_row + pedestrian_pathway_row
            update_pedestrian_col = pedestrian_col + pedestrian_pathway_col
            updated_pedestrian_loc.append((update_pedestrian_row, update_pedestrian_col))
        return updated_pedestrian_loc

class StreetLightMaker():
    def __init__(self, n_rows, n_cols, sl_coverage, md_coverage, action): # action will later be deleted ?
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.sl_coverage = sl_coverage
        self.md_coverage = md_coverage
        self.action = action # later will be deleted
        self.list_sl_loc = self.create_sl(self.n_rows, self.n_cols, self.sl_coverage)
        self.grid_sl = self.create_sl_grid(self.n_rows, self.n_cols, self.list_sl_loc)
        self.list_sl_cvrg_area = self.create_sl_coverage_area(self.n_rows, self.n_cols, self.sl_coverage, self.list_sl_loc)
        self.list_md_cvrg_area = self.create_motion_detection_area(self.n_rows, self.n_cols, self.md_coverage, self.list_sl_loc)
        self.grid_sl_brightness = self.brightness_coverage(self.action, self.n_rows, self.n_cols, self.list_sl_cvrg_area)

    @staticmethod
    def create_sl(n_rows, n_cols, sl_coverage):
        n_rows = n_rows - 1
        n_cols = n_cols - 1
        list_sl_rows = list(range(sl_coverage, n_rows, (sl_coverage*2+1)))
        list_sl_cols = list(range(sl_coverage, n_cols, (sl_coverage*2+1)))
        
        if list_sl_rows == []:
            list_sl_rows = [0]
            
        if list_sl_cols == []:
            list_sl_cols = [0]

        list_sl_loc = []
        for row, column in product(list_sl_rows, list_sl_cols):
            list_sl_loc.append((row, column))
        
        return list_sl_loc # return a list of tuple [street light coordinate]

    @staticmethod
    def create_sl_grid(n_rows, n_cols, list_sl_loc):
        grid_sl = []
        for idx_row in range(n_rows):
            row = []
            for idx_column in range(n_cols):
                row.append(0)
            grid_sl.append(row)
        for idx_sl in range(len(list_sl_loc)):
            sl_loc = list_sl_loc[idx_sl]
            grid_sl[sl_loc[ROW]][sl_loc[COLUMN]] = 1
        return grid_sl # return the grid and each street light position on the grid

    @staticmethod
    def create_sl_coverage_area(n_rows, n_cols, sl_coverage, list_sl_loc):
        list_sl_cvrg_area = []
        for idx_sl in range(len(list_sl_loc)):
            sl_loc = list_sl_loc[idx_sl]
            sl_row = sl_loc[ROW]
            sl_col = sl_loc[COLUMN]
            coverage_row = list(range((sl_row - sl_coverage), (sl_row + sl_coverage + 1)))
            coverage_col = list(range((sl_col - sl_coverage), (sl_col + sl_coverage + 1)))
            rows = list(range(0, n_rows))
            cols = list(range(0, n_cols))
            coverage_row = list(x for x in coverage_row if x in rows)
            coverage_col = list(x for x in coverage_col if x in cols)
            coverage_coor = []
            for row, col in product(coverage_row, coverage_col):
                coverage_coor.append((row, col))
            list_sl_cvrg_area.append(coverage_coor)
        return list_sl_cvrg_area # return list of list of tuple

    @staticmethod
    def create_motion_detection_area(n_rows, n_cols, md_coverage, list_sl_loc):
        list_md_cvrg_area = []
        for idx_sl in range(len(list_sl_loc)):
            sl_loc = list_sl_loc[idx_sl]
            sl_row = sl_loc[ROW]
            sl_col = sl_loc[COLUMN]
            coverage_row = list(range((sl_row - md_coverage), (sl_row + md_coverage + 1)))
            coverage_col = list(range((sl_col - md_coverage), (sl_col + md_coverage + 1)))
            rows = list(range(0, n_rows))
            cols = list(range(0, n_cols))
            coverage_row = list(x for x in coverage_row if x in rows)
            coverage_col = list(x for x in coverage_col if x in cols)
            coverage_coor = []
            for row, col in product(coverage_row, coverage_col):
                coverage_coor.append((row, col))
            list_md_cvrg_area.append(coverage_coor)
        return list_md_cvrg_area # return list of list of tuple
        
    @staticmethod
    def brightness_coverage(action, n_rows, n_cols, list_sl_cvrg_area):
        grid_sl_brightness = []
        assert len(action) == len(list_sl_cvrg_area)
        for idx_row in range(n_rows):
            row = []
            for idx_column in range(n_cols):
                row.append(0)
            grid_sl_brightness.append(row)
        for idx_action in range(len(action)):
            sl_cvrg = list_sl_cvrg_area[idx_action]
            sl_action = action[idx_action]
            for idx_cvrg in range(len(sl_cvrg)):
                cvrg_coor = sl_cvrg[idx_cvrg]
                cvrg_row = cvrg_coor[ROW]
                cvrg_col = cvrg_coor[COLUMN]
                if sl_action == 1: # will be generalized
                    grid_sl_brightness[cvrg_row][cvrg_col] = 1
                elif sl_action == 2: # will be generalized
                    grid_sl_brightness[cvrg_row][cvrg_col] = 2
                else  : # will be generalized
                    grid_sl_brightness[cvrg_row][cvrg_col] = 3
        return grid_sl_brightness # return grid for brightness

class Agent():
    def _init_():
        pass

    def action():
        pass

class Environment():

    def __init__(self, n_rows, n_cols, list_position, num_of_pedestrian, sl_coverage, md_coverage, action):
        self.time = 0
        self.list_position = list_position
        self.num_of_pedestrian = num_of_pedestrian
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.sl_coverage = sl_coverage
        self.md_coverage = md_coverage
        self.action = action # will be removed
        self.PM_class = PedestriansMaker(self.n_rows, self.n_cols, self.list_position, self.num_of_pedestrian)
        self.SLM_class = StreetLightMaker(self.n_rows, self.n_cols, self.sl_coverage, self.md_coverage, self.action)
        self.list_current_pedestrian = self.PM_class.list_pedestrian_s.copy()

    @staticmethod
    def obs(list_sl_cvrg_area, list_current_pedestrian):
        list_pedestrian_detected = []
        for idx_sl in range(len(list_sl_cvrg_area)):
            sl_cvrg_area = list_sl_cvrg_area[idx_sl]
            test_coor = 0
            for idx_pedestrian in range(len(list_current_pedestrian)):
                current_pedestrian_loc = list_current_pedestrian[idx_pedestrian]
                pedestrian_count = sl_cvrg_area.count((current_pedestrian_loc[ROW], current_pedestrian_loc[COLUMN]))
                test_coor = test_coor + pedestrian_count
            if test_coor == 0:
                list_pedestrian_detected.append(test_coor)
            else:
                list_pedestrian_detected.append(1)
        return list_pedestrian_detected
    
    @staticmethod
    def reward(action, list_sl_cvrg_area, list_current_pedestrian):
        reward = []
        for idx_sl in range(len(action)):
            sl_cvrg_area = list_sl_cvrg_area[idx_sl]
            sl_action = action[idx_sl]
            test_coor = 0
            for idx_pedestrian in range(len(list_current_pedestrian)):
                current_pedestrian_loc = list_current_pedestrian[idx_pedestrian]
                pedestrian_count = sl_cvrg_area.count((current_pedestrian_loc[ROW], current_pedestrian_loc[COLUMN]))
                test_coor = test_coor + pedestrian_count
                if test_coor == 0: 
                    test_coor = 1
                elif test_coor > 3: # will be generalized
                    test_coor = 3
            if sl_action == test_coor:
                reward.append(0)
            elif sl_action < test_coor:
                reward.append(sl_action - test_coor)
            else:
                reward.append(test_coor - sl_action)
        return reward

    @staticmethod
    def done(list_current_pedestrian, list_pedestrian_d):
        status = []
        for idx_pedestrian in range(len(list_current_pedestrian)):
            current_pedestrian_loc = list_current_pedestrian[idx_pedestrian]
            pedestrian_d = list_pedestrian_d[idx_pedestrian]
            if current_pedestrian_loc == pedestrian_d:
                status.append(True)
            else:
                status.append(False)
        done = all(status)
        return done

    @staticmethod
    def info():
        pass

    def reset(self):
        self.time = 0
        self.PM_class = PedestriansMaker(self.n_rows, self.n_cols, self.list_position, self.num_of_pedestrian)
        self.SLM_class = StreetLightMaker(self.n_rows, self.n_cols, self.sl_coverage, self.md_coverage, self.action)
        self.list_current_pedestrian = self.PM_class.list_pedestrian_s.copy()
        list_pedestrian_detected = self.obs(self.SLM_class.list_sl_cvrg_area, self.list_current_pedestrian)
        return list_pedestrian_detected

    def step(self):
        self.time = self.time + 1
        # using self in the list_current_pedestrian will overwrite the value in _init_? is the changes need to overwrite init?
        self.list_current_pedestrian = self.PM_class.pedestrian_loc_update(self.list_current_pedestrian, self.PM_class.list_pedestian_pathway)
        list_pedestrian_detected = self.obs(self.SLM_class.list_sl_cvrg_area, self.list_current_pedestrian)
        done = self.done(self.list_current_pedestrian, self.PM_class.list_pedestrian_d)
        reward = self.reward(self.action, self.SLM_class.list_sl_cvrg_area, self.list_current_pedestrian)
        info = {
            'curent pedestrian loc' : self.list_current_pedestrian
        }
        return list_pedestrian_detected, done, reward, info # info will be added later

    # def render(self):
    #     print(self.grid_pedestrian)

print('test for environment')
action = [1,3]
c = Environment(n_rows, n_cols, list_position, num_of_pedestrian, sl_coverage, md_coverage, action)
print('reset = {}'.format(c.reset()))
print('pedestrian starting point : {}'.format(c.list_current_pedestrian))
while c.done(c.list_current_pedestrian, c.PM_class.list_pedestrian_d) is False:
    obs, done, reward, info = c.step()
    print('obs : {}'.format(obs))
    print('done : {}'.format(done))
    print('reward : {}'.format(reward))
    print('info : {}'.format(info))


test for environment
reset = [1, 1]
pedestrian starting point : [(0, 0), (0, 9), (0, 9)]
obs : [1, 1]
done : False
reward : [0, 0]
info : {'curent pedestrian loc': [(0, 1), (0, 8), (0, 8)]}
obs : [1, 1]
done : False
reward : [0, 0]
info : {'curent pedestrian loc': [(0, 2), (0, 7), (0, 7)]}
obs : [1, 1]
done : False
reward : [0, 0]
info : {'curent pedestrian loc': [(0, 3), (0, 6), (0, 6)]}
obs : [1, 1]
done : False
reward : [0, 0]
info : {'curent pedestrian loc': [(0, 4), (0, 5), (0, 5)]}
obs : [1, 1]
done : False
reward : [-1, -1]
info : {'curent pedestrian loc': [(0, 5), (0, 4), (0, 4)]}
obs : [1, 1]
done : False
reward : [-1, -1]
info : {'curent pedestrian loc': [(0, 6), (0, 3), (0, 3)]}
obs : [1, 1]
done : False
reward : [-1, -1]
info : {'curent pedestrian loc': [(0, 7), (0, 2), (0, 2)]}
obs : [1, 1]
done : False
reward : [-1, -1]
info : {'curent pedestrian loc': [(0, 8), (0, 1), (0, 1)]}
obs : [1, 1]
done : True
reward : [-1, -1]
info : {'curent pedestrian loc': [(0, 9), (0, 0), (0

In [29]:
print('test for pedestrian maker')
a = PedestriansMaker(n_rows, n_cols, list_position, num_of_pedestrian)
print('starting destination combination = {}'.format(a.create_starting_destination(list_position)))
print('pedestrian starting destination combination = {}'.format(a.create_pedestrians_sd(num_of_pedestrian, a.list_sd)))
print('pedestrian starting destination combination = {}'.format(a.list_pedestrian_sd))
print('pedestrian starting = {}'.format(a.list_pedestrian_s))
print('pedestrian destination = {}'.format(a.list_pedestrian_d))
print('pedestrian location starting = {}'.format(a.create_location_grid(n_rows, n_cols, num_of_pedestrian, a.list_pedestrian_s)))
print('pedestrian pathway = {}'.format(a.list_pedestian_pathway))

test for pedestrian maker
starting destination combination = [((0, 0), (0, 9)), ((0, 9), (0, 0))]
pedestrian starting destination combination = [[(0, 9), (0, 9), (0, 0)], [(0, 0), (0, 0), (0, 9)]]
pedestrian starting destination combination = [[(0, 9), (0, 9), (0, 0)], [(0, 0), (0, 0), (0, 9)]]
pedestrian starting = [(0, 9), (0, 9), (0, 0)]
pedestrian destination = [(0, 0), (0, 0), (0, 9)]
pedestrian location starting = [[1, 0, 0, 0, 0, 0, 0, 0, 0, 2]]
pedestrian pathway = [(0, -1), (0, -1), (0, 1)]


In [ ]:
print('test for street light')
action = [2,3]
b = StreetLightMaker(n_rows, n_cols, sl_coverage, md_coverage, action)
print('list of street light coordinate = {}'.format(b.list_sl_loc))
print('street light on grid = {}'.format(b.grid_sl))
print('coverage coordinate for each light = {}'.format(b.list_sl_cvrg_area))
print('grid of brightness = {}'.format(b.grid_sl_brightness))
print('motion detection coverage area = {}'.format(b.list_md_cvrg_area))

In [ ]:
import random
from itertools import product
random.seed(1)

# indexing:
STARTING = 0
DESTINATION = 1
ROW = 0
COLUMN = 1

# variables:
n_rows = 1
n_cols = 10
list_position = [(0,0), (0,9)]
num_of_pedestrian = 3
sl_coverage = 2
md_coverage = 2
brightness_level = (1, 2, 3)

class Agent:

    def _init_(self, n_rows, n_cols, list_position, num_of_pedestrian, sl_coverage, md_coverage):
        self.brightness_level = brightness_level
        self.list_md_cvrg_area = self.StreetLightMaker_class.list_md_cvrg_area
        self.pedestrian_loc = 
        self.list_brightness = self.action(self.list_md_cvrg_area, self.pedestrian_loc, self.brightness_level)
        
    def action(list_md_cvrg_area, pedestrian_loc, brightness_level):
        list_brightness = []
        for idx_sl in range(len(list_md_cvrg_area)):
            sl_md_cvrg = list_md_cvrg_area[idx_sl]
            test_coor = sl_md_cvrg.count((pedestrian_loc[ROW], pedestrian_loc[COLUMN]))
            if test_coor == 0:
                brightness = 0
            else:
                brightness = random.choice(brightness_level)
            list_brightness.append(brightness)
        return  

            